In [1]:
import pandas as pd
import numpy as np

urlItemList = "https://github.com/ProTraitInfra/Item-lists/blob/main/Lists/Definitieve%20CRF's%20OpenClinica/Generic/Generic%202/PROTRAIT_GENERIC.xls?raw=true"
df = pd.read_excel(urlItemList,sheet_name="Items", usecols=["ITEM_NAME*", "DESCRIPTION_LABEL*", "RESPONSE_OPTIONS_TEXT", "RESPONSE_VALUES_OR_CALCULATIONS"])

#rename column names
df = df.rename(columns={'ITEM_NAME*': 'ItemName', 'DESCRIPTION_LABEL*': 'ItemDescription', 'RESPONSE_OPTIONS_TEXT':'ValueStringList', 'RESPONSE_VALUES_OR_CALCULATIONS' : "ValueIntegerList" })
#change all NaN to none values
df = df.replace({np.nan: None})
#remove comma's in valueStringList column within categorical values
df['ValueStringList']=df.ValueStringList.replace({"\\\\, ": "-"},regex=True)

print(df)

                 ItemName                                    ItemDescription  \
0        GEN_TREAT_CENTRE                         Particle (treating) centre   
1        GEN_REFER_CENTRE                                   Referring centre   
2        GEN_DATE_REGISTR   Date of registration (first visit RT department)   
3              GEN_GENDER                                    Gender by birth   
4                 GEN_AGE                                   Age at diagnosis   
5          GEN_YEAR_BIRTH                                      Year of birth   
6           GEN_EDUCATION                                  Educational level   
7            GEN_RELATION                                Relationship status   
8             GEN_SMOKING                                     Smoking status   
9       GEN_SM_PACKYEARS1  If patient is a past smoker, number of pack years   
10      GEN_SM_PACKYEARS2  If patient is a current smoker, number of pack...   
11      GEN_SM_STOPMONTHS  If patient is

In [2]:
from string import Template

#create a class for continous variables. extract columns from df and enter in the template shacle
class CreateTermMappingValueTriples:
    def __init__(self, ItemName, OntologyClass, ItemDescription, ValueStringList, ValueIntegerList):
        """
        The CreateTermMappingValueTriples class creates the triples needed to select values for the categorical variables in the term mapper .

        Input parameters:
            self:  
            ItemName = ProTRAIT name of the item 
            ItemDescription = human readable description of the variable
            OntologyClass = ontology uri of the item from the r2rml file
            ValueStringList = list of possible values in string format
            ValueStringList = list of corresponding integer values for libre clinica
        """
        self.ItemName = ItemName
        self.OntologyClass = OntologyClass
        self.ItemDescription = ItemDescription
        self.ValueStringList = ValueStringList
        self.ValueIntegerList = ValueIntegerList

    def triples(self):
        """
        Triples function  creates the triples needed to select values for the categorical variables in the term mapper .
        """
        
        #add optional rangeMax shacle shape
        ValueTriples = ""
        ValueList = []
        triples = []
        
        #creating lists from the value option columns
        stringList = self.ValueStringList.split(",")
        integerList = self.ValueIntegerList.split(",")
        
        #removing spaces in the list
        stringList = [item.lstrip() for item in stringList]
        stringList = [item.replace(' ', '_') for item in stringList]
        print("#", stringList, integerList)

        #raise exception if something went wrong    
        if len(stringList) != len(integerList): raise Exception("Unequeal lengths of string and integer lists in " + self.ItemName) 

        #create triples     
        for string, integer in zip(stringList, integerList):
            triples = Template("""
                            protrait:${string} 
                                rdfs:subClassOf ${ontologyClass}; 
                                protrait:LC_Code "${integer}"^^xsd:integer.
                            
                            """).substitute(string=string, ontologyClass=self.OntologyClass, integer=integer)
        
            ValueTriples += triples

        return ValueTriples    
        

In [3]:
import rdflib #import graph, Literal
import sparql_dataframe as sd
import SPARQLWrapper
from tabulate import tabulate
import re


graph = rdflib.Graph()
#graph.parse("https://raw.githubusercontent.com/MaastrichtU-CDS/protrait_mapping-unifications/master/GenericList2/GenericList2_stagingDB.ttl", format='ttl')
graph.parse("https://gitlab.com/MatthijsSloep/r2rml_mappings/-/raw/main/GenericList2_stagingDB.ttl", format='ttl')


results = graph.query("""
PREFIX rr: <http://www.w3.org/ns/r2rml#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ncit: <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#>
PREFIX roo: <http://www.cancerdata.org/roo/>

SELECT ?templateURI ?ontologyClass
WHERE {
       ?s rdf:type rr:TriplesMap;
       		rr:subjectMap [
            rr:template ?templateURI;
            rr:class ?ontologyClass;].
	    }""")

      

columnNames = [str(item) for item in results.vars]
df2 = pd.DataFrame(results, columns=columnNames)

#print(tabulate(df2, headers = 'keys', tablefmt = 'github'))

#extract the proTRAIT item name from the uri 

for index, row in df2.iterrows():
       oldVal = row['templateURI']
       res = re.findall(r'\{.*?\}', row['templateURI'])

       #selects last item in list
       value = res[-1]
       #strips brackets
       result = value[1:-1]
       
       #print(oldVal,result)
       row['templateURI'] = str(row['templateURI']).replace(oldVal,result)
      


#create dictionary with prefixes and uri's
ontologyDict = {
    "http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#" : "ncit:",
    "http://www.cancerdata.org/roo/":"roo:", 
    "http://purl.bioontology.org/ontology/STY/" : "sty:", 
    "http://semanticscience.org/resource/SIO_" : "sio:", 
    "http://purl.bioontology.org/ontology/ICD10/" : "icd:", 
    "http://purl.obolibrary.org/obo/UO_" : "uo:",
    "http://www.w3.org/2006/time#" : "time"
}

#loop over dictionary to replace values in df
for key, value in ontologyDict.items():
    df2['ontologyClass'] = df2['ontologyClass'].str.replace(key,value, regex=False)
    
    
print(tabulate(df2, headers = 'keys', tablefmt = 'github'))

|    | templateURI           | ontologyClass                                                 |
|----|-----------------------|---------------------------------------------------------------|
|  0 | id                    | ncit:C16960                                                   |
|  1 | id                    | ncit:C25364                                                   |
|  2 | gen_treat_centre      | ncit:C19326                                                   |
|  3 | gen_date_registr      | ncit:C25646                                                   |
|  4 | gen_refer_centre      | ncit:C19326                                                   |
|  5 | gen_gender            | ncit:C28421                                                   |
|  6 | gen_age               | roo:C100002                                                   |
|  7 | gen_year_birth        | ncit:C25155                                                   |
|  8 | gen_education         | ncit:C17953        

In [6]:
#create a proTRAITTermmapperValueTriples.ttl file  
#f = open("proTRAITTermmapperValueTriples.ttl", "w")

def getOntologyClass(protraitItemName, df):
    
    ontologyClass = ''

    for index, value in enumerate(df['templateURI']):                 
        if value == protraitItemName:
            ontologyClass = df['ontologyClass'].iloc[index]
            
    return ontologyClass

#write triples
for i, row in df.iterrows():
    if row['ValueIntegerList'] != None :
        name = row['ItemName'].lower() 
        ontoClass = getOntologyClass(name, df2)
        
        print("#", name, "-", ontoClass)
        if ontoClass != "":
            genericTriples = CreateTermMappingValueTriples(
                row.ItemName, ontoClass, row.ItemDescription, row.ValueStringList, row.ValueIntegerList
                )
        #f.write(valueTriples.triples())
            print(genericTriples.triples())


# gen_treat_centre - ncit:C19326
# ['Not_applicable_(in_case_of_photon_RT)', 'GPTC_Groningen', 'HollandPTC_Delft', 'MAASTRO_protonen', 'APTC_Amsterdam'] ['0', '1', '2', '3', '4']

                            protrait:Not_applicable_(in_case_of_photon_RT) 
                                rdfs:subClassOf ncit:C19326; 
                                protrait:LC_Code "0"^^xsd:integer.
                            
                            
                            protrait:GPTC_Groningen 
                                rdfs:subClassOf ncit:C19326; 
                                protrait:LC_Code "1"^^xsd:integer.
                            
                            
                            protrait:HollandPTC_Delft 
                                rdfs:subClassOf ncit:C19326; 
                                protrait:LC_Code "2"^^xsd:integer.
                            
                            
                            protrait:MAASTRO_protonen 
                    